In [1]:
from llm_call import process_markdown_files_in_folder
from pathlib import Path


{'description': 'Do not be lazy while working on your tasks!', 'properties': {'thought_process': {'description': "Go step-by-step through your previous analysis, picking out the most influencal points for a nuanced classification.Focus on capturing unique emotional patterns specific to the user's behavior, avoiding generic statements that could apply to anyone. Utilize specific examples from the user's comments to highlight key emotional tendencies.", 'title': 'Thought Process', 'type': 'string'}, 'nuanced_classification': {'description': "Provide a nuanced classification based on prior analysis, ensuring the identification of distinct emotionalexpressions unique to the user. Avoid universal characterizations that lack specificity. Use emotion labels appropriately, acknowledging their dependency on context, and back your classification with precise examples from the user's comments.", 'title': 'Nuanced Classification', 'type': 'string'}, 'discussion_behaviour': {'description': "Analyze

In [2]:
import uuid


random_uuid = uuid.uuid4()
result = process_markdown_files_in_folder()


--- Start of User Processing ---


Processing files: 0it [00:00, ?it/s]

START ANALYSIS OF USER:  8177
Output matches schema ordering exactly!


Processing files: 1it [00:31, 31.88s/it]

START ANALYSIS OF USER:  19792
Output matches schema ordering exactly!


Processing files: 2it [01:04, 32.37s/it]

--- End of User Processing ---


In [5]:
#print(result) #needs to be tranformed into a python dict. @copilot

for r in result:
    #print(r)
    print(r["model-Step 2: Summarization"])


The user's emotional expressions online are dynamically constructed, reflecting interplay between core affect, cognitive appraisals, and sociocultural context.  Initial responses to the Semenya case exhibited high negative valence and arousal, stemming from appraisals of injustice and perceived threats to fairness in women's sports.  These reactions were tempered by later attempts at rule-based solutions, suggesting a shift towards more controlled emotional regulation.  However, high arousal persisted in interactions involving significant disagreement or controversial topics.  Comments on disaster preparedness showed a different emotional profile, characterized by increasing anxiety and a focus on practical concerns.  Overall, the user demonstrates emotional reactivity, influenced by the online forum’s dynamics and their existing beliefs, reflecting a complex and context-dependent emotional construction process.  A binary understanding of gender appears prevalent in their initial frami

In [7]:
from langsmith import Client
from dotenv import load_dotenv

load_dotenv()

client = Client()
dataset_name = "Example Dataset"

# Filter runs to add to the dataset
runs = client.list_runs(
  project_name="LLM-Classification-Pipeline",
  run_type="chain",
  is_root=True,
  filter='has(tags, "default_dataset")',
  error=False,
)
# Iterate over the retrieved runs and print the desired fields
# Check if runs are empty and print the contents
if not runs:
    print("No runs found matching the criteria.")
else:
    for run in runs:
        print(run.prompt_tokens)
        print(run.completion_tokens)
        print(run.total_tokens)
        print(run.tags[2])
        print(run.id)
        print(run.feedback_stats)
        #print(run.outputs["model-Step 2: Summarization"])

13933
1674
15607
default_dataset
7ba13a7f-6c46-4894-a380-5e89aec72bda
None
28204
1686
29890
default_dataset
6df8c388-ac6a-40e9-aaf8-b94633a49ae2
None
13828
1603
15431
default_dataset
8c401a60-3672-4fe1-a5fd-45c38375d021
None
13685
1426
15111
default_dataset
920c3324-806b-4c7f-bac0-8451c5fedbf3
None
30512
1769
32281
default_dataset
f7ed6bd7-ba57-4b81-9e4f-3149d0628abd
{'hallucination': {'n': 2, 'avg': 1.0, 'stdev': 0.0, 'errors': 0, 'show_feedback_arrow': True, 'values': {}}, 'judgement': {'n': 0, 'avg': None, 'stdev': None, 'errors': 0, 'show_feedback_arrow': True, 'values': {"The student's answer provides an analysis of the user's emotional state and cognitive appraisals based on the context_sphere. The context_sphere includes comments made by the user UNBEQUEM, which are indeed critical and express frustration towards various topics such as politics, banks, and media. The student's interpretation of UNBEQUEM's affective state as negative and skeptical is supported by the context_sphe

In [4]:
from langsmith import Client
from dotenv import load_dotenv
import json

load_dotenv()

client = Client()
dataset_name = "default_dataset_iterative_development_runs"

# Filter runs to add to the dataset
runs = client.list_runs(
  project_name="LLM-Classification-Pipeline",
  run_type="chain",
  is_root=True,
  filter='has(tags, "default_dataset")',
  error=False,
)
# Iterate over the retrieved runs and print the desired fields
# Check if runs are empty and print the contents
"""
if not runs:
    print("No runs found matching the criteria.")
else:
    for run in runs:
        print(run.outputs["model-Step 1: Classification"])
        print(run.outputs["model-Step 2: Summarization"])
"""
DELETE_DATASET = True

if DELETE_DATASET:
  if client.has_dataset(dataset_name=dataset_name):
      client.delete_dataset(dataset_name=dataset_name)
      print("Deleted existing dataset")
  dataset = client.create_dataset(
      dataset_name=dataset_name,
      description="Default Dataset for Langsmith Development runs",
      inputs_schema={
  "type": "object",
  "title": "dataset_input_schema",
  "required": [
    "step_1_classification"
  ],
  "properties": {
    "step_1_classification": {
      "type": "object"
    }
  }
}
      ,
      outputs_schema={
  "type": "object",
  "title": "dataset_output_schema",
  "required": [
    "step_2_classification_summary"
  ],
  "properties": {
    "step_2_classification_summary": {
      "type": "string"
    }
  }
})

else:
  dataset=client.read_dataset(dataset_name=dataset_name)



#output = json.loads(output) if isinstance(output, str) else output
#print(run.id)

#dataset = client.create_dataset(dataset_name, description="An example dataset")
for run in runs:
  input= run.outputs["model-Step 1: Classification"]
  output=run.outputs["model-Step 2: Summarization"]

  input = json.loads(input) if isinstance(input, str) else input
  client.create_example(
      inputs={"step_1_classification": input},
      outputs={"step_2_classification_summary": output},
      metadata={"user_id": run.tags[1]},
      created_at=run.end_time,
      dataset_id=dataset.id,
      split="dev",
      example_id=run.id,
      source_run_id=run.id,
      
  )



Deleted existing dataset


In [ ]:

client = Client()
dataset_id = "f9ce2b77-2f54-4f3e-bc20-ad46d34ef303"  # Replace with your actual dataset ID
dataset = client.get_dataset(dataset_id)
share_link = dataset.share_link
print(share_link)

AttributeError: 'Client' object has no attribute 'get_dataset'